In [23]:
import duckdb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import json

In [24]:
# Connect to an in-memory DuckDB instance
con = duckdb.connect(database=':memory:')

In [25]:
# Load CSV files
train_df = con.execute("SELECT * FROM 'imdb/train-*.csv'").fetchdf()
directors_df = con.execute("SELECT * FROM 'imdb/directing.json'").fetchdf()
writers_df = con.execute("SELECT * FROM 'imdb/writing.json'").fetchdf()

# Load rotten tomatoes CSV files
rotten_movies = con.execute("SELECT * FROM 'rotten_tomatoes_movies.csv'").fetchdf()

In [26]:
# Convert JSON-like strings to actual dictionaries
def safe_json_loads(x):
	try:
		return json.loads(x) if isinstance(x, str) and x else x
	except json.JSONDecodeError:
		return None

directors_df['movie'] = directors_df['movie'].apply(safe_json_loads)
directors_df['director'] = directors_df['director'].apply(safe_json_loads)

writers_df['movie'] = writers_df['movie'].apply(safe_json_loads)
writers_df['writer'] = writers_df['writer'].apply(safe_json_loads)

# Explode dictionaries into separate rows
directors_expanded = directors_df.explode('movie').explode('director')
directors_expanded.rename(columns={'movie': 'tconst', 'director': 'director_id'}, inplace=True)

writers_expanded = writers_df.explode('movie').explode('writer')
writers_expanded.rename(columns={'movie': 'tconst', 'writer': 'writer_id'}, inplace=True)

# Check the cleaned data
print(directors_expanded.head())
print(writers_expanded.head())

  tconst director_id
0      0           0
0      0           1
0      0           2
0      0           3
0      0           4
  tconst writer_id
0   None      None
1   None      None
2   None      None
3   None      None
4   None      None


In [27]:
# Create base tables in DuckDB from pandas DataFrames
con.execute("CREATE TABLE train_df AS SELECT * FROM train_df")
con.execute("CREATE TABLE directors_expanded AS SELECT * FROM directors_expanded")
con.execute("CREATE TABLE writers_expanded AS SELECT * FROM writers_expanded")

# rotten tomatoes update
con.execute("CREATE TABLE rotten_movies AS SELECT * FROM rotten_movies")

# Create indexes for faster lookup
con.execute("CREATE INDEX train_df_tconst_idx ON train_df (tconst);")
con.execute("CREATE INDEX directors_expanded_tconst_idx ON directors_expanded (tconst);")
con.execute("CREATE INDEX writers_expanded_tconst_idx ON writers_expanded (tconst);")

# rotten tomatoes update
con.execute("CREATE INDEX rotten_movies_rotten_tomatoes_link ON rotten_movies (rotten_tomatoes_link);")

# Join tables
movies_df = con.execute("""
    SELECT t.*, d.director_id, w.writer_id
    FROM train_df t
    LEFT JOIN directors_expanded d ON t.tconst = d.tconst
    LEFT JOIN writers_expanded w ON t.tconst = w.tconst
""").fetchdf()

final_movies_df = con.execute(""" 
    SELECT m.*, r.tomatometer_rating, r.tomatometer_count, r.audience_rating, r.audience_count
    FROM movies_df m
    LEFT JOIN rotten_movies r ON m.primaryTitle = r.movie_title
""").fetchdf()

In [28]:
# Check the final dataset
print(final_movies_df.head())

   column0     tconst                                       primaryTitle  \
0     5326  tt0814255  Percy Jackson & the Olympians: The Lightning T...   
1     4584  tt0381849                                       3:10 to Yuma   
2     4593  tt0382628                                         Dark Water   
3      298  tt0033873                                           Man Hunt   
4      189  tt0028773                                           Dead End   

  originalTitle startYear endYear runtimeMinutes  numVotes  label  \
0          None      2010      \N            118  183678.0  False   
1          None        \N    2007            122  300411.0   True   
2          None      2005      \N            105   61511.0  False   
3      Man Hunt        \N    1941            105    5231.0   True   
4      Dead End      1937      \N             93       NaN   True   

   director_id  writer_id  tomatometer_rating  tomatometer_count  \
0         <NA>       <NA>                  49               

In [29]:
final_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8224 entries, 0 to 8223
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   column0             8224 non-null   int64  
 1   tconst              8224 non-null   object 
 2   primaryTitle        8224 non-null   object 
 3   originalTitle       4112 non-null   object 
 4   startYear           8224 non-null   object 
 5   endYear             8224 non-null   object 
 6   runtimeMinutes      8224 non-null   object 
 7   numVotes            7406 non-null   float64
 8   label               8224 non-null   bool   
 9   director_id         0 non-null      Int32  
 10  writer_id           0 non-null      Int32  
 11  tomatometer_rating  2735 non-null   Int64  
 12  tomatometer_count   2735 non-null   Int64  
 13  audience_rating     2730 non-null   Int64  
 14  audience_count      2730 non-null   Int64  
dtypes: Int32(2), Int64(4), bool(1), float64(1), int64(1), o

In [30]:
# Convert startYear to integer and then to movie age
final_movies_df['startYear'] = pd.to_numeric(final_movies_df['startYear'], errors='coerce')
final_movies_df['endYear'] = pd.to_numeric(final_movies_df['endYear'], errors='coerce')

# If startYear is empty or NaN, fill it with endYear
final_movies_df['startYear'].fillna(final_movies_df['endYear'], inplace=True)

# If primaryTitle is empty or NaN, fill it with originalTitle
final_movies_df['primaryTitle'].fillna(final_movies_df['originalTitle'], inplace=True)

# Remove the columns originalTitle and endYear
final_movies_df.drop(columns=['originalTitle', 'endYear'], inplace=True)

final_movies_df['movie_age'] = 2025 - final_movies_df['startYear']

# Log-transform numVotes
final_movies_df['log_numVotes'] = np.log1p(final_movies_df['numVotes'])

# Categorize runtime
final_movies_df['runtime_category'] = pd.cut(
    pd.to_numeric(movies_df['runtimeMinutes'], errors='coerce'),
    bins=[0, 60, 90, 120, 180, np.inf],
    labels=['short', 'medium', 'long', 'very long', 'epic']
)

final_movies_df.to_csv("cleaned_imdb_data.csv", index=False)

C:\Users\oisin\AppData\Local\Temp\ipykernel_2276\1990886512.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_movies_df['startYear'].fillna(final_movies_df['endYear'], inplace=True)
C:\Users\oisin\AppData\Local\Temp\ipykernel_2276\1990886512.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always 

In [31]:
# Load data
df = pd.read_csv("cleaned_imdb_data.csv")

# Prepare features and labels
X = df[['movie_age', 'log_numVotes', 'tomatometer_rating', 'tomatometer_count', 'audience_rating', 'audience_count']]  # Add more features as needed
y = df['label']

In [32]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.6954407294832827


In [34]:
### TEST SUBMISSION ###

# Load test data
test_df = con.execute("SELECT * FROM 'imdb/test_hidden.csv'").fetchdf()

# Convert startYear to numeric
test_df['startYear'] = pd.to_numeric(test_df['startYear'], errors='coerce')

# Apply same feature transformations
test_df['movie_age'] = 2025 - test_df['startYear']
test_df['log_numVotes'] = np.log1p(test_df['numVotes'])

# Predict
X_test_hidden = test_df[['movie_age', 'log_numVotes']]
test_df['predicted_label'] = clf.predict(X_test_hidden)

### VALIDATION SUBMISSION ###
# Load test data
val_df = con.execute("SELECT * FROM 'imdb/validation_hidden.csv'").fetchdf()

# Convert startYear to numeric
val_df['startYear'] = pd.to_numeric(val_df['startYear'], errors='coerce')

# Apply same feature transformations
val_df['movie_age'] = 2025 - val_df['startYear']
val_df['log_numVotes'] = np.log1p(val_df['numVotes'])

# Predict
X_val_hidden = val_df[['movie_age', 'log_numVotes']]
val_df['predicted_label'] = clf.predict(X_val_hidden)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- audience_count
- audience_rating
- tomatometer_count
- tomatometer_rating


In [ ]:
# Save for submission
test_df[['predicted_label']].to_csv("submission_test.csv", index=False, header=False)

# Save for submission
validation_df[['predicted_label']].to_csv("submission_validation.csv", index=False, header=False)